In [101]:
import sys
import os
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import re
import subprocess

In [102]:
epochs = 80
trainable = True

In [103]:
from keras_contrib.layers import CRF


In [104]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [105]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [106]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [76]:

# preproecessing data from Conll
def get_sentences(filename):
    '''
        -DOCSTART- -X- -X- O

    EU NNP B-NP B-ORG
    rejects VBZ B-VP O
    German JJ B-NP B-MISC
    call NN I-NP O
    to TO B-VP O
    boycott VB I-VP O
    British JJ B-NP B-MISC
    lamb NN I-NP O
    . . O O
    
    '''
    
    '''
    read file
    return format :
    [ ['EU', 'B-ORG'], ['rejects', 'O'], ['German', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['British', 'B-MISC'], ['lamb', 'O'], ['.', 'O'] ]
    '''
    f = open(filename,'rb')
    sentences = []
    sentence = []
    for line in f:
        splits = line.split()
        try:
            word=splits[0].decode()
            if word=='-DOCSTART-':
                continue
            label=splits[-1].decode()
            temp=[word,label]
            sentence.append(temp)
        except Exception as e:
            if len(sentence)!=0:
                sentences.append(sentence)
                sentence=[]
    return sentences



In [107]:
train_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [77]:
# conll
train_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.train')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testa')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/CONLL/deu/deu_utf.testb')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


12152
2867
3005


In [108]:
print(train_sentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [109]:
# train_sentences = modify_labels(train_sentences)
# dev_sentences = modify_labels(dev_sentences)
# test_sentences = modify_labels(test_sentences)

In [110]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [111]:
# do lowercasing of words for fasttext embeddings. Not for others.
labelSet = set()
words = {}
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token, char, label in sentence:
            token =token
            for c in char:
                characters.add(c)
            labelSet.add(label)
#             words[token.lower()] = True
#             words[token]=True
            words[re.sub('\d','#',token)] = True

In [112]:
print(labelSet)

{'O', 'B-LOC', 'I-LOC', 'B-ORGpart', 'I-OTHderiv', 'I-ORGpart', 'B-LOCderiv', 'I-ORGderiv', 'B-LOCpart', 'B-PERderiv', 'B-OTHpart', 'B-ORGderiv', 'I-OTH', 'I-PER', 'B-PERpart', 'I-PERpart', 'I-ORG', 'I-LOCpart', 'B-ORG', 'B-OTHderiv', 'I-OTHpart', 'B-PER', 'I-PERderiv', 'B-OTH', 'I-LOCderiv'}


In [113]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [114]:
print(label2Idx)

{'B-OTHpart': 10, 'O': 0, 'I-LOC': 2, 'B-PERpart': 14, 'B-ORGpart': 3, 'I-ORGpart': 5, 'B-LOC': 1, 'I-OTHderiv': 4, 'B-LOCderiv': 6, 'I-PERpart': 15, 'I-ORG': 16, 'B-ORG': 18, 'I-ORGderiv': 7, 'B-LOCpart': 8, 'B-OTHderiv': 19, 'I-OTHpart': 20, 'B-PER': 21, 'B-ORGderiv': 11, 'I-PERderiv': 22, 'I-OTH': 12, 'B-OTH': 23, 'I-LOCderiv': 24, 'I-LOCpart': 17, 'I-PER': 13, 'B-PERderiv': 9}


In [115]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [42]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/home/jindal/notebooks/fastText/german_words_word2vecf.txt','wb')
f.write(string_words.encode())
f.close()

In [116]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'PADDING_TOKEN': 7, 'contains_digit': 6, 'numeric': 0, 'mainly_numeric': 5, 'allUpper': 2, 'other': 4, 'initialUpper': 3, 'allLower': 1}


In [ ]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('german_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word.lower()]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [ ]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_word2vecf.embeddings", encoding="utf-8")

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0] in words:
#         print('yes')
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [117]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_word2vec.embeddings", 'rb')

for line in fEmbeddings:
    split = line.strip().split()
    word = split[0].decode()
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(300) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, 300)
        wordEmbeddings.append(vector)

    if word in words:
        
#         for num in split[1:]:
#             print(float(num))
        vector = np.array([float(num.decode()) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [118]:
print(word2Idx)

{'Zweitstimmen': 31693, 'Besitzungen': 4497, 'ewiges': 35651, 'Unterburg': 34674, 'Auftrieb': 17484, 'Rückfragen': 25630, 'Magazines': 42166, 'Tiedge': 55258, 'Prinzipiell': 19581, 'alsbald': 16801, 'Bock': 10869, 'medizinischen': 4698, 'Schritt': 2927, 'Holzspielzeug': 53907, 'Hogarth': 37379, 'reüssierte': 44830, 'Strache': 37719, 'Nimm': 34456, 'Gewichtsverlagerung': 51410, 'Schlitzrüssler': 57358, 'Sandoy': 56914, 'Einzeltäter': 51111, 'Eisenach': 6947, 'Hanging': 48567, 'Hört': 35234, 'Damenwelt': 53413, 'Windows': 3079, 'erloschenen': 27421, 'ausschöpfen': 45448, 'Rheinische': 14847, 'Lainz': 44887, 'nachgefragt': 24851, 'selbstbestimmte': 49358, 'Branchenverbände': 56172, 'Finkelstein': 41292, 'büßt': 54125, 'Brasserie': 45655, 'ziemlicher': 31547, 'Schottinnen': 55946, 'Schwimmbad': 19757, 'Linde': 10314, 'Haager': 12150, 'Hochzeiten': 18413, 'zweien': 32360, 'Turmtriebwagen': 58369, 'Gruppensieger': 13172, 'Zeitsprünge': 56269, 'geringeren': 8277, 'Tocqueville': 48147, 'love':

In [ ]:
word2Idx={}
wordEmbeddings=[]

fEmbeddings = open("/home/jindal/notebooks/embeddings/embeddings/model_GloVe_w6.embeddings", encoding="utf-8")

for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
#     print(word)
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0] in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[word] = len(word2Idx)
wordEmbeddings=np.array(wordEmbeddings)

In [119]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings))
print(len(wordEmbeddings[0]))
print(len(wordEmbeddings[2]))

60615
300
300


In [120]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'í': 4, 'P': 1, 'O': 2, '!': 3, 'κ': 5, '~': 285, 'Π': 6, 'г': 7, '樓': 9, 'h': 0, 'έ': 10, 'r': 11, 'є': 14, '−': 288, 'أ': 15, 'н': 61, 'Ł': 17, 'w': 18, 'オ': 19, 'У': 20, 'ť': 22, 'î': 63, 'U': 24, 'ū': 125, 'Þ': 25, 'Ø': 291, 'm': 27, 'Ц': 256, 'µ': 29, 'Ž': 31, 'ð': 293, '학': 186, '3': 32, 'o': 324, 'ô': 35, 'λ': 37, 'ж': 36, '\x95': 40, '±': 38, '·': 39, 'ĩ': 46, 'D': 41, 'ý': 42, 'р': 44, '²': 45, 'π': 47, '´': 8, 'ř': 48, '造': 49, 'M': 50, 'û': 225, 'â': 52, '5': 77, '⊃': 321, '½': 54, 'l': 55, 'j': 299, '£': 174, 't': 58, '«': 60, 'ы': 62, 'ν': 128, 'A': 65, 'ñ': 66, 'ċ': 57, 'ς': 290, 'v': 67, 'ć': 68, 'ň': 71, '‹': 70, 'ض': 72, 'И': 74, 'n': 75, 'å': 78, '寝': 240, '[': 303, '太': 16, 'ź': 80, 'ø': 133, '-': 81, 'ë': 82, 'б': 83, 'ї': 85, 'я': 134, 'β': 86, 'ł': 87, 'ế': 222, ']': 100, 'ő': 89, 'η': 94, 'a': 91, 'а': 92, '→': 73, '\x92': 95, 'ι': 242, 'σ': 142, 'Λ': 96, '\x96': 97, '+': 98, 'č': 99, '–': 300, '›': 105, '&': 103, '▪': 106, '公': 283, '†': 264, '“': 108, 'ψ': 107

In [121]:
print(trainSentences[0]) 

[['Schartau', ['S', 'c', 'h', 'a', 'r', 't', 'a', 'u'], 'B-PER'], ['sagte', ['s', 'a', 'g', 't', 'e'], 'O'], ['dem', ['d', 'e', 'm'], 'O'], ['"', ['"'], 'O'], ['Tagesspiegel', ['T', 'a', 'g', 'e', 's', 's', 'p', 'i', 'e', 'g', 'e', 'l'], 'B-ORG'], ['"', ['"'], 'O'], ['vom', ['v', 'o', 'm'], 'O'], ['Freitag', ['F', 'r', 'e', 'i', 't', 'a', 'g'], 'O'], [',', [','], 'O'], ['Fischer', ['F', 'i', 's', 'c', 'h', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['"', ['"'], 'O'], ['in', ['i', 'n'], 'O'], ['einer', ['e', 'i', 'n', 'e', 'r'], 'O'], ['Weise', ['W', 'e', 'i', 's', 'e'], 'O'], ['aufgetreten', ['a', 'u', 'f', 'g', 'e', 't', 'r', 'e', 't', 'e', 'n'], 'O'], [',', [','], 'O'], ['die', ['d', 'i', 'e'], 'O'], ['alles', ['a', 'l', 'l', 'e', 's'], 'O'], ['andere', ['a', 'n', 'd', 'e', 'r', 'e'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['überzeugend', ['ü', 'b', 'e', 'r', 'z', 'e', 'u', 'g', 'e', 'n', 'd'], 'O'], ['war', ['w', 'a', 'r'], 'O'], ['"', ['"'], 'O'], ['.', ['.'], 'O']]


In [122]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences, word2Idx,  label2Idx, case2Idx, char2Idx))
dev_set = padding(createMatrices(devSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx, char2Idx))

In [123]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print((train_set[0][2]))

[[56272, 2573, 25, 9, 21310, 9, 79, 6779, 3, 2272, 275, 9, 8, 47, 1053, 10800, 3, 7, 872, 234, 28, 16151, 35, 9, 2], [3, 1, 1, 4, 3, 4, 1, 3, 4, 3, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 1, 1, 4, 4], array([[195, 284,   0, ...,   0,   0,   0],
       [116,  91, 274, ...,   0,   0,   0],
       [111, 310,  27, ...,   0,   0,   0],
       ...,
       [ 18,  91,  11, ...,   0,   0,   0],
       [144,   0,   0, ...,   0,   0,   0],
       [181,   0,   0, ...,   0,   0,   0]], dtype=int32), [21, 0, 0, 0, 18, 0, 0, 0, 0, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
25
[[195 284   0 ...   0   0   0]
 [116  91 274 ...   0   0   0]
 [111 310  27 ...   0   0   0]
 ...
 [ 18  91  11 ...   0   0   0]
 [144   0   0 ...   0   0   0]
 [181   0   0 ...   0   0   0]]


In [124]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [125]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [126]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10496       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_31 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_34 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [127]:
print(i)

100


In [129]:
for x in range(1):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
#             predLabels = np.concatenate(predLabels).ravel()
#             correctLabels = np.concatenate(correctLabels).ravel()
#             f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
            print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
#             print(f1_dev)
    
            if f1_dev > maxf1:
                model.save('german_ner_vec_germeval.h5')
                maxf1 = f1_dev

    

Epoch 0/80
55/51 [================================] - 13s 232ms/step
Epoch 1/80
55/51 [================================] - 14s 249ms/step
Epoch 2/80
55/51 [================================] - 13s 233ms/step
Epoch 3/80
55/51 [================================] - 13s 240ms/step
Epoch 4/80
55/51 [================================] - 13s 233ms/step
Epoch 5/80
55/51 [================================] - 14s 252ms/step
Epoch 6/80
55/51 [================================] - 13s 234ms/step
Epoch 7/80
55/51 [================================] - 13s 228ms/step
Epoch 8/80
55/51 [================================] - 14s 250ms/step
Epoch 9/80
55/51 [================================] - 13s 236ms/step
Epoch 10/80
55/51 [================================] - 14s 248ms/step
Epoch 11/80
55/51 [================================] - 13s 235ms/step
Epoch 12/80
55/51 [================================] - 13s 245ms/step
Epoch 13/80
55/51 [================================] - 13s 236ms/step
Epoch 14/80
55/51 [===========

2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.831, Rec: 0.773, F1: 0.801
Epoch 59/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.818, Rec: 0.793, F1: 0.805
Epoch 60/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.836, Rec: 0.765, F1: 0.799
Epoch 61/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.808, Rec: 0.791, F1: 0.799
Epoch 62/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.823, Rec: 0.785, F1: 0.803
Epoch 63/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.798, Rec: 0.799, F1: 0.799
Epoch 64/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.757, Rec: 0.804, F1: 0.780
Epoch 65/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.818, Rec: 0.770, F1: 0.793
Epoch 66/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.799, Rec: 0.801, F1: 0.800
Epoch 67/80
2198/2199 [========

In [ ]:
print(id2word)

In [ ]:
id2word = {v: k for k, v in word2Idx.items()}
print(len(correctLabels))        

model.load_weights('/home/jindal/notebooks/jindal/NER/german_ner_vec_germeval.h5')
predLabels, correctLabels = tag_dataset(test_batch)        
# pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
# print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_word2vec_germeval'+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
#         string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        f.write(string)
        id+=1

command = "perl nereval.perl"
directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
# directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
# command = "perl conlleval"
input_file= open(input_file_name)
output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_word2vec_germeval'+'.txt'
output_file=open(output_file_name,'w')
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_germeval
                          )
out, err = process.communicate()
#     print(out)
print('******************************************************************************')

2199
5098/5099 [============================>.] - ETA: 0s

In [ ]:
import os
os.getcwd()